In [33]:
# importo librerias
import ast
from datetime import datetime
import pickle
from collections import namedtuple
from functools import wraps
import logging

import pandas as pd
import numpy as np
from scipy.stats import entropy

from meli_pull_apidata import MeliApiClient

In [125]:
# Decorators

def log_shape(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.shape))
        return result
    return wrapper

def log_dtypes(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.dtypes))
        return result
    return wrapper

#### Funciones Útiles

In [126]:
# util functions
def read_dict(name_file):
    with open(name_file, 'rb') as f:
        dic = pickle.load(f)
    return dic

def dict_topandas(dict_file, _type=1):
    return pd.json_normalize(
            [dict_file[k] 
             if dict_file.get(k) else 0 
             for k in dict_file],
            sep='_'
            )
def parse_timestamp(text):
    """
    try:
        text_s = str(text).split('T')[0]
        return datetime.strptime(text_s, '%Y-%m-%d')
    except:
        return np.nan
    """
    try:
        return pd.to_datetime(text)
    except ValueError as e:
        return np.nan

def time2datetime_converter(col):
    timepart = (df[col].astype(str)
                   .str.replace('\.0$', '')  # NaNs force float dtype
                   .str.pad(4, fillchar='0'))
    return pd.to_datetime(dff[col] + ' ' +
                           timepart.str.slice(0, 2) + ':' +
                           timepart.str.slice(2, 4),
                           errors='coerce')

def parse_boolean(col):
    try:
        return [1 if type(val)==str and val.lower.strip() == 'true' 
                else 0 
                for val in col]
    except: 
        pass
   
# pandas df functions
@log_shape
@log_dtypes
def general_process_columns(df_in, columns_nt):
    colname_mapper = {src:dest for src, dest, _ in columns_nt}
    for col in columns_nt:
        try:
            df_in[col.src] = [col.convert(val) 
                          if val == val and val is not None else np.nan 
                          for val in df_in[col.src]]
        except ValueError as e:
            print(col.src, e)
        except KeyError as ke:
            print(col.src, ke)
        #except TypeError as te:
        #    print(col.src, te)
    return df_in.rename(columns=colname_mapper)


@log_shape
@log_dtypes
def parse_tags(df_in, coltag):
    nest = pd.get_dummies(
                df_in[coltag].apply(pd.Series),
                prefix='attr'
                )
    lcol = nest.columns.tolist()
    scol = set()
    scol.update(lcol)
    for att_col in scol:
        df_in[att_col] = nest[att_col].T.sum()
                              
    return df_in

@log_shape
@log_dtypes
def parse_prices(df_in, col='prices_prices'):
    def count_prom(lines):
        if type(lines) == 'list':
            count = 0
            for line in lines:
                if line.get('type') == 'promotion':
                    count += 1
            return count
        return 0
    df_in[col] = list(map(count_prom, df_in[col]))
    return df_in        



In [116]:
Column = namedtuple('Column', 'src dest convert')
sites_schema = [
    Column('id', 'id', str),
    Column('title', 'pub_title', str),
    Column('seller_registration_date',
           'seller_registration_date', parse_timestamp),
    #Column('seller_tags', 'seller_tags', pd.Categorical),
    Column('seller_seller_reputation_level_id', 
           'seller_reputation_level', pd.Categorical),
    Column('seller_seller_reputation_power_seller_status', 
           'power_seller_status', pd.Categorical),
    #Column('seller_seller_reputation_transactions_completed',
    #       'seller_transactions_completed', int),
    Column('seller_seller_reputation_transactions_ratings_negative', 
           'seller_ratings_negative', int),
    Column('seller_seller_reputation_transactions_ratings_neutral', 
           'seler_ratings_neutral', int),
    Column('seller_seller_reputation_transactions_ratings_positive', 
           'seller_ratings_positive', int),
    Column('seller_seller_reputation_transactions_total', 
           'seller_transactions_total', int),
    Column('seller_seller_reputation_metrics_sales_completed', 
           'seller_sales_completed', int),
    Column('seller_seller_reputation_metrics_claims_value', 
           'seller_claims_value', int),
    Column('seller_seller_reputation_metrics_claims_rate', 
           'seller_claims_rate', float),
    Column('seller_seller_reputation_metrics_cancellations_rate', 
           'seller_cancellations_rate', float),
    Column('seller_seller_reputation_metrics_cancellations_value',
           'seller_cancellations_value', int),
    Column('seller_seller_reputation_metrics_delayed_handling_time_value',
           'seller_delayed_handling_value', int),
    Column('seller_seller_reputation_metrics_delayed_handling_time_rate',
           'seller_delayed_handling_rate', float),
    Column('sale_price', 'sale_price', float),
    Column('buying_mode', 'buying_mode', pd.Categorical),
    Column('listing_type_id', 'listing_type', pd.Categorical),
    Column('condition', 'condition', pd.Categorical),
    #Column('accepts_mercadopago', 'accepts_mercadopago', parse_boolean),
    Column('installments_quantity', 'installments_quantity', int),
]
items_schema = [
    Column('id', 'id', str),
    Column('base_price', 'base_price', float),
    Column('initial_quantity', 'initial_quantity', int), 
    Column('available_quantity', 'available_quantity', int),
    Column('start_time', 'start_time', parse_timestamp),
    Column('stop_time', 'stop_time', parse_timestamp),
    Column('sold_quantity', 'sold_quantity', int), 
    #Column('international_delivery_mode', 'international_delivery_mode', parse_boolean), 
    Column('status', 'status', parse_boolean),
    # Column('warranty', 'warranty',parse_text) 
    Column('date_created', 'date_created', parse_timestamp),
    #Column('date_updated', 'date_updated', parse_timestamp),
    Column('health', 'health', float),
    Column('site_id', 'site_id', pd.Categorical),
    Column('price', 'price', float),
    Column('currency_id', 'currency_id', pd.Categorical),
    Column('available_quantity', 'available_quantity', int),
    Column('sold_quantity', 'sold_quantity', int),
    Column('accepts_mercadopago', 'accepts_mercadopago', parse_boolean),
    Column('original_price', 'original_price', float),
    Column('category_id', 'category_id', pd.Categorical),
    Column('domain_id', 'domain_id', pd.Categorical),
    Column('catalog_listing', 'catalog_listing', parse_boolean),
    Column('seller_id', 'seller_id', int),
    Column('shipping_free_shipping', 'shipping_free_shipping', parse_boolean),
    Column('shipping_mode', 'shipping_mode', pd.Categorical),
    Column('seller_address_state_name', 'seller_address_state_name', pd.Categorical),
    Column('seller_contact_webpage', 'seller_contact_webpage', str)
    ]

In [94]:
list_columns_drop_from_sites = [
    'attributes', 'tags', 'prices_prices'
    ]

In [121]:
sites_column_list = [x.dest for x in sites_schema]
items_column_list = [x.dest for x in items_schema]
master_column_list = sites_column_list + items_column_list
master_column_list

['id',
 'pub_title',
 'seller_registration_date',
 'seller_reputation_level',
 'power_seller_status',
 'seller_ratings_negative',
 'seler_ratings_neutral',
 'seller_ratings_positive',
 'seller_transactions_total',
 'seller_sales_completed',
 'seller_claims_value',
 'seller_claims_rate',
 'seller_cancellations_rate',
 'seller_cancellations_value',
 'seller_delayed_handling_value',
 'seller_delayed_handling_rate',
 'sale_price',
 'buying_mode',
 'listing_type',
 'condition',
 'installments_quantity',
 'id',
 'base_price',
 'initial_quantity',
 'available_quantity',
 'start_time',
 'stop_time',
 'sold_quantity',
 'status',
 'date_created',
 'health',
 'site_id',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'accepts_mercadopago',
 'original_price',
 'category_id',
 'domain_id',
 'catalog_listing',
 'seller_id',
 'shipping_free_shipping',
 'shipping_mode',
 'seller_address_state_name',
 'seller_contact_webpage']

Resultados de la api en local - no en repo -

In [6]:
ls results/*

results/meli_items_api_data.p
results/meli_items_metrics_api_data_new.p
results/meli_items_metrics_api_data.p
results/meli_sites_api_data.p
results/meli_users_metrics_api_data_new.p
results/meli_users_metrics_api_data.p

results/old:
meli_items_api_data.p  meli_sites_api_data_2021-03-07T22_22_38.p


In [7]:
# A DF
sites = read_dict('results/meli_sites_api_data.p')
items = read_dict('results/meli_items_api_data.p')
# Quedan asi para ser integrados en la construccion #build 
item_metrics = read_dict('results/meli_items_metrics_api_data.p')
user_metrics = read_dict('results/meli_users_metrics_api_data.p')
len(sites), len(items), len(item_metrics), len(user_metrics)

(28059, 28059, 6690, 2686)

### Build Dataset

In [8]:
sites_df = dict_topandas(sites)
items_df = dict_topandas(items)

In [9]:
id2attr = {_id: len(d) 
           for _id, d 
           in zip(items_df['id'], 
                  items_df['attributes'])}

In [122]:
df = (sites_df
      .pipe(parse_tags, 'tags')
      .pipe(parse_prices, 'prices_prices')
      .pipe(general_process_columns, sites_schema)
      .drop(list_columns_drop_from_sites, axis=1)
      .merge(items_df.pipe(
                  general_process_columns,
                  items_schema),
             on='id', how='inner', suffixes=("_remove", None))
      # seleccionamos las columnas
      [master_column_list]
      .assign(n_attributes=list(
          map(lambda x: id2attr.get(x, 0), items_df.id))
          )
      .assign(item_visits=list(
          map(lambda x: item_metrics.get(x, 0), items_df.id))
          )
      .assign(user_visits=list(
          map(lambda x: user_metrics.get(x, 0), items_df.seller_id))
          )
      )
df.head()

,id,pub_title,seller_registration_date,seller_reputation_level,power_seller_status,seller_ratings_negative,seler_ratings_neutral,seller_ratings_positive,seller_transactions_total,seller_sales_completed,...,domain_id,catalog_listing,seller_id,shipping_free_shipping,shipping_mode,seller_address_state_name,seller_contact_webpage,n_attributes,item_visits,user_visits
0,MLA871657598,Smart Tv Rca X55andtv Led 4k 55 100v/240v,2016-10-04 15:46:28-04:00,"['5_green'] Categories (1, object): ['5_green']","['platinum'] Categories (1, object): ['platinum']",0.0,0.0,0.0,12283.0,1965.0,...,"['MLA-TELEVISIONS'] Categories (1, object): ['...",NaN,230379131,NaN,"['not_specified'] Categories (1, object): ['no...","['Buenos Aires'] Categories (1, object): ['Bue...",NaN,30,34,438015
1,MLA885144534,Smart Tv Philips 6600 Series 50pud6654/77 Led ...,2016-10-04 15:46:28-04:00,"['5_green'] Categories (1, object): ['5_green']","['platinum'] Categories (1, object): ['platinum']",0.0,0.0,0.0,12283.0,1965.0,...,"['MLA-TELEVISIONS'] Categories (1, object): ['...",NaN,230379131,NaN,"['me2'] Categories (1, object): ['me2']","['Buenos Aires'] Categories (1, object): ['Bue...",NaN,33,352,438015
2,MLA910600162,Smart Tv Samsung Series 7 Un43tu7000gczb Led 4...,2013-07-31 08:14:50-04:00,"['3_yellow'] Categories (1, object): ['3_yellow']",NaN,0.0,0.0,0.0,86020.0,12295.0,...,"['MLA-TELEVISIONS'] Categories (1, object): ['...",NaN,143125485,NaN,"['custom'] Categories (1, object): ['custom']","['Capital Federal'] Categories (1, object): ['...",NaN,36,1071,1369576
3,MLA895877679,Tv Box Noga Pc Ultra Estándar 4k 8gb Negro C...,2002-04-13 02:00:00-04:00,"['5_green'] Categories (1, object): ['5_green']","['platinum'] Categories (1, object): ['platinum']",0.0,0.0,0.0,19134.0,2956.0,...,"['MLA-STREAMING_MEDIA_DEVICES'] Categories (1,...",NaN,8963146,NaN,"['me2'] Categories (1, object): ['me2']","['Buenos Aires'] Categories (1, object): ['Bue...",NaN,29,1528,116539
4,MLA877436109,Smart Tv Tedge Ntv504k Led 4k 50 220v,2019-03-19 08:41:57-04:00,"['5_green'] Categories (1, object): ['5_green']","['platinum'] Categories (1, object): ['platinum']",0.0,0.0,0.0,367677.0,60652.0,...,"['MLA-TELEVISIONS'] Categories (1, object): ['...",NaN,417972986,NaN,"['me2'] Categories (1, object): ['me2']","['Buenos Aires'] Categories (1, object): ['Bue...",NaN,58,44250,3444269


In [123]:
df.describe()

,seller_ratings_negative,seler_ratings_neutral,seller_ratings_positive,seller_transactions_total,seller_sales_completed,seller_claims_value,seller_claims_rate,seller_cancellations_rate,seller_cancellations_value,seller_delayed_handling_value,...,available_quantity,sold_quantity,accepts_mercadopago,original_price,catalog_listing,seller_id,shipping_free_shipping,n_attributes,item_visits,user_visits
count,27924.000000,27924.000000,27924.000000,27924.000000,27923.000000,27922.000000,27923.000000,27922.000000,27922.000000,27922.000000,...,28059.000000,28059.000000,0.0,2364.000000,0.0,2.805900e+04,0.0,28059.000000,28059.000000,2.805900e+04
mean,0.000680,0.022669,0.130784,32696.628384,5079.386133,59.852804,0.009071,0.004441,27.035993,65.850082,...,299.062974,39.313518,NaN,40351.025503,NaN,1.748124e+08,NaN,23.789622,328.402509,3.566324e+05
std,0.026076,0.148848,0.337170,69152.530189,12070.311034,136.104633,0.020125,0.025054,75.786377,265.123849,...,3104.685402,190.831605,NaN,55593.693434,NaN,1.488736e+08,NaN,25.316738,3084.779232,7.941920e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,68.000000,NaN,5.923900e+04,NaN,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000,866.000000,178.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,NaN,4590.000000,NaN,6.362895e+07,NaN,11.000000,0.000000,5.314000e+03
50%,0.000000,0.000000,0.000000,7769.000000,1100.000000,9.000000,0.007900,0.000000,2.000000,3.000000,...,1.000000,4.000000,NaN,17523.500000,NaN,1.448441e+08,NaN,18.000000,0.000000,9.648400e+04
75%,0.000000,0.000000,0.000000,28413.000000,3960.000000,50.000000,0.013200,0.005500,15.000000,26.000000,...,1.000000,15.000000,NaN,56993.000000,NaN,2.368620e+08,NaN,28.000000,0.000000,3.105100e+05
max,1.000000,1.000000,1.000000,455592.000000,90772.000000,942.000000,1.000000,1.000000,642.000000,3678.000000,...,50000.000000,5000.000000,NaN,606149.000000,NaN,7.242827e+08,NaN,224.000000,226965.000000,7.880053e+06


In [124]:
df.columns.tolist()

['id',
 'pub_title',
 'seller_registration_date',
 'seller_reputation_level',
 'power_seller_status',
 'seller_ratings_negative',
 'seler_ratings_neutral',
 'seller_ratings_positive',
 'seller_transactions_total',
 'seller_sales_completed',
 'seller_claims_value',
 'seller_claims_rate',
 'seller_cancellations_rate',
 'seller_cancellations_value',
 'seller_delayed_handling_value',
 'seller_delayed_handling_rate',
 'sale_price',
 'buying_mode',
 'listing_type',
 'condition',
 'installments_quantity',
 'id',
 'base_price',
 'initial_quantity',
 'available_quantity',
 'start_time',
 'stop_time',
 'sold_quantity',
 'status',
 'date_created',
 'health',
 'site_id',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'accepts_mercadopago',
 'original_price',
 'category_id',
 'domain_id',
 'catalog_listing',
 'seller_id',
 'shipping_free_shipping',
 'shipping_mode',
 'seller_address_state_name',
 'seller_contact_webpage',
 'n_attributes',
 'item_visits',
 'user_visits']

In [129]:
acolsel = ['seller_delayed_handling_rate',
 'sale_price',
 'buying_mode',
 'listing_type',
 'condition',
 'installments_quantity']
colsel = ['id',
 'base_price',
 'initial_quantity',
 'available_quantity',
 'start_time',
 'stop_time',
 'sold_quantity',
 'status',
 'date_created',
 'health',
 'site_id',
 'price',
 'currency_id']
df[colsel].head()

,id,id,base_price,initial_quantity,available_quantity,available_quantity,start_time,stop_time,sold_quantity,sold_quantity,status,date_created,health,site_id,price,currency_id
0,MLA871657598,MLA871657598,53100.0,15,1,1,2020-08-04 15:27:00+00:00,2040-07-30 04:00:00+00:00,5,5,NaN,2020-08-04 15:27:00+00:00,NaN,"['MLA'] Categories (1, object): ['MLA']",53100.0,"['ARS'] Categories (1, object): ['ARS']"
1,MLA885144534,MLA885144534,46200.0,24,1,1,2020-10-23 18:12:47+00:00,2040-10-18 04:00:00+00:00,5,5,NaN,2020-10-23 18:12:47+00:00,NaN,"['MLA'] Categories (1, object): ['MLA']",46200.0,"['ARS'] Categories (1, object): ['ARS']"
2,MLA910600162,MLA910600162,57999.0,19,1,1,2021-03-04 21:27:00+00:00,2041-02-27 04:00:00+00:00,5,5,NaN,2021-03-04 21:27:01+00:00,NaN,"['MLA'] Categories (1, object): ['MLA']",57999.0,"['ARS'] Categories (1, object): ['ARS']"
3,MLA895877679,MLA895877679,4099.0,230,1,1,2020-11-05 15:33:52+00:00,2040-10-31 04:00:00+00:00,200,200,NaN,2020-11-05 15:33:52+00:00,NaN,"['MLA'] Categories (1, object): ['MLA']",4099.0,"['ARS'] Categories (1, object): ['ARS']"
4,MLA877436109,MLA877436109,45999.0,2134,500,500,2020-09-08 17:52:37+00:00,2040-09-03 12:11:36+00:00,500,500,NaN,2020-09-08 17:52:38+00:00,NaN,"['MLA'] Categories (1, object): ['MLA']",45999.0,"['ARS'] Categories (1, object): ['ARS']"


In [28]:
mac = MeliApiClient()
things = set(mac.SITES_Q_1)
things.add('tv')
things.add('disco')
things.add('discoexterno')
things.add('consola')

In [44]:
def bag_words_parse(cell):
    global things
    set_in = set(cell.lower().split(' '))
    inter = things.intersection(set_in)
    return ' '.join(inter)

In [45]:
sites_df['parsed_titles'] = list(map(lambda x:bag_words_parse(x), sites_df.title))

In [46]:
sites_df.parsed_titles.unique()

array(['tv', '', 'tv laptop', 'tv consola', 'celular tv',
       'playstation tv', 'tv proyector', 'proyector', 'disco tv',
       'microondas', 'ventilador', 'phone', 'celular', 'celular phone',
       'cargador phone', 'celular tv phone', 'cargador',
       'celular cargador phone', 'celular auriculares',
       'celular cargador', 'celular tv proyector', 'consola',
       'tv proyector laptop', 'laptop', 'playstation', 'mochila', 'disco',
       'disco consola', 'playstation disco', 'disco laptop', 'kindle',
       'cargador kindle', 'amplificador', 'amplificador auriculares',
       'auriculares amplificador', 'consola amplificador',
       'parlante amplificador', 'parlante', 'celular amplificador',
       'auriculares', 'cargador amplificador', 'tv amplificador',
       'auriculares proyector', 'parlante proyector',
       'parlante tv proyector', 'proyector laptop',
       'ventilador proyector', 'mochila cargador', 'mochila laptop',
       'mochila playstation', 'mochila celula

In [52]:
sites_df[sites_df.parsed_titles == ''].title.head(1).tolist()

[' Xiaomi Mi Box S Mdz-22-ab  De Voz 4k 8gb  Negro Con Memoria Ram De 2gb']

In [49]:
sites_df[sites_df.parsed_titles == 'mochila aspiradora'].title.head()

14988       Aspiradora Mochila Polvo Gamma 1200w 4 Litros 
15001    Aspiradora Inalambrica Makita Mochila 18v Dvc2...
15036            Aspiradora Mochila 4lts La-5002m Lusqtoff
15054    Aspiradora Mochila Gamma 1200w 4 Litros 1,6 Hp...
15493    Aspiradora De Mochila Gamma 4lts En Seco 1200w...
Name: title, dtype: object

In [50]:
things

{'aireacondicionado',
 'amplificador',
 'aspiradora',
 'aspiradorarobot',
 'auriculares',
 'auto%toyota',
 'bicicleta',
 'cablehdmi',
 'cargador',
 'celular',
 'consola',
 'consola%yamaha',
 'disco',
 'disco%externo',
 'discoexterno',
 'drone',
 'estufa',
 'extractortablet',
 'heladera',
 'hidrolavadora',
 'kindle',
 'laptop',
 'microondas',
 'mochila',
 'parlante',
 'parrillaagas',
 'phone',
 'playstation',
 'proyector',
 'sintetizador',
 'termotanque',
 'thermomix',
 'tv',
 'tv%204k',
 'ventilador'}

In [91]:
for cc in sites_df.columns:
    if 'conditions' in cc.lower():
        print(cc)